## An Automated Portfolio Trading System with Feature Preprocessing and Recurrent Reinforcement Learning

### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.data import Data, get_buy_and_hold_returns, INDICATORS

from lib.model import RRL, train, validation

from lib.backtest import (
    run_rrl_strategy, 
    create_backtest_dataset, 
    generate_layout, 
    generate_nrows, 
    generate_subplot_titles, 
    init_subplots, 
    populate_subplots,
    plot_cumulative_profits, 
    generate_df_barplot, 
    make_cumrets_barplot,
)

from lib.metrics import (
    calc_cumulative_profits, 
    calc_sharpe_ratio, 
)

In [3]:
import numpy as np 
import pandas as pd

from pickle import load, dump

### Data

#### Settings

In [683]:
# assets in the portfolio
ASSETS = ["XOM", "VZ", "NKE", "AMAT", "MCD", "MSFT", "AAP", "NOV"]
n_assets = 4
assets = np.random.choice(ASSETS, size=n_assets, replace=False).tolist()

# date settings
start_date = "2009-12-31"
end_date = "2017-12-29"
window_size = 100

# number of principal components (None means no PCA)
pca_ncp = None

# technical indicators
momentum = [
    "MOM",
    "MACD",
    "MFI",
    "RSI"
]

volatility = [
    "ATR",
    "NATR"
]

cycle = [ 
    # "HTDCP",
    # "HTS",
    # "HTTMMM"
]

volume = [
    # "CO",
    "OBV"
]

indicators = momentum + volatility + cycle + volume
indicators

['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'OBV']

#### Instantiate `Data` object

In [684]:
data = Data(start_date, end_date, window_size, assets, indicators, pca_ncp)
data

[*********************100%***********************]  4 of 4 completed


Data(start_date='2009-12-31', end_date='2017-12-29', window_size=100, assets=['MSFT', 'AMAT', 'VZ', 'AAP'], indicators=['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'OBV'], pca_ncp=None)

In [685]:
buy_and_hold_returns = get_buy_and_hold_returns(data.df)

### `RRL` - Training and validation

In [686]:
initial_invest = 100
n_epochs = 50

#### No transaction fees 

$\delta=0$

In [687]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features) 
print(rrl) 

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_no_fees_5ncp.pkl"
file_path = "./backup/no_pca/rrl_no_fees_3.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=7, delta=0, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  14%|█▍        | 7/50 [00:00<00:00, 44.90it/s, sharpe_ratio=0.3]  


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 0.5697483146029468
Cumulative profits: 105.04881392004461
Training window: 2010-06-30/2010-10-14


Training in progress...:  44%|████▍     | 22/50 [00:00<00:00, 52.09it/s, sharpe_ratio=0.353]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 0.7792604644375536
Cumulative profits: 110.78395694571498
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: 0.06856908245760629
Cumulative profits: 309.0583098977286
Training window: 2017-03-31/2017-07-18


Training in progress...:  14%|█▍        | 7/50 [00:00<00:01, 39.31it/s, sharpe_ratio=0.00461]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 1.2476046073497487
Cumulative profits: 361.0043713470738
Training window: 2017-08-23/2017-12-07


Training in progress...:   6%|▌         | 3/50 [00:00<00:01, 32.61it/s, sharpe_ratio=1.11]

Less than 30 observations in the validation set after preprocessing.


#### Transactions fees 

$\delta \in \{0.001, 0.003, 0.005\}$

In [688]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.001) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_10bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_10bps_3.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=7, delta=0.001, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  46%|████▌     | 23/50 [00:00<00:00, 48.63it/s, sharpe_ratio=-.0921]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 0.6313766388963883
Cumulative profits: 105.98017331715735
Training window: 2010-06-30/2010-10-14


Training in progress...:  52%|█████▏    | 26/50 [00:00<00:00, 49.34it/s, sharpe_ratio=0.507]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 0.06004145380085566
Cumulative profits: 106.16232568319316
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: 0.09669680785279135
Cumulative profits: 222.95156319662436
Training window: 2017-03-31/2017-07-18


Training in progress...:   4%|▍         | 2/50 [00:00<00:01, 25.98it/s, sharpe_ratio=0.0535]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 1.4900023114271206
Cumulative profits: 273.694134527547
Training window: 2017-08-23/2017-12-07


Training in progress...:  10%|█         | 5/50 [00:00<00:01, 35.86it/s, sharpe_ratio=1.5]


Less than 30 observations in the validation set after preprocessing.


In [689]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.003) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_30bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_30bps_3.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=7, delta=0.003, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:   6%|▌         | 3/50 [00:00<00:01, 42.84it/s, sharpe_ratio=-.297]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 0.6517647731480847
Cumulative profits: 105.72857081036344
Training window: 2010-06-30/2010-10-14


Training in progress...:  46%|████▌     | 23/50 [00:00<00:00, 35.87it/s, sharpe_ratio=0.677]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 0.09387579983774401
Cumulative profits: 106.1872066640627
Training window: 2010-11-19/2011-03-09


...

Training in progress...:   2%|▏         | 1/50 [00:00<00:01, 28.13it/s, sharpe_ratio=-.601]

Validation window: 2017-08-23/2017-12-07


Sharpe ratio on validation set: 0.7606161631813793
Cumulative profits: 153.41027452064293
Training window: 2017-08-23/2017-12-07


Training in progress...:   4%|▍         | 2/50 [00:00<00:01, 34.04it/s, sharpe_ratio=0.759]

Less than 30 observations in the validation set after preprocessing.


In [690]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.005) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

# file_path = "./backup/pca/rrl_30bps_5ncp.pkl"
file_path = "./backup/no_pca/rrl_50bps_3.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=4, n_features=7, delta=0.005, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  24%|██▍       | 12/50 [00:00<00:00, 49.45it/s, sharpe_ratio=-.4]  


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 0.4337637199833513
Cumulative profits: 103.52020832732309
Training window: 2010-06-30/2010-10-14


Training in progress...:   8%|▊         | 4/50 [00:00<00:01, 43.79it/s, sharpe_ratio=0.486]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: -0.07087314700610653
Cumulative profits: 102.59394490712504
Training window: 2010-11-19/2011-03-09


...

Sharpe ratio on validation set: -1.832995687577808
Cumulative profits: 70.94455359801641
Training window: 2017-03-31/2017-07-18


Training in progress...:  12%|█▏        | 6/50 [00:00<00:01, 36.46it/s, sharpe_ratio=-1.98]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: -0.7678167670246479
Cumulative profits: 66.57395802296112
Training window: 2017-08-23/2017-12-07


Training in progress...:   2%|▏         | 1/50 [00:00<00:01, 28.72it/s, sharpe_ratio=-.768]

Less than 30 observations in the validation set after preprocessing.


### Backtest

In [691]:
file_names = [f"rrl_{fees}_3" for fees in ("no_fees", "10bps", "30bps", "50bps")]

path = "./backup/no_pca/"
results = {
    f.replace("results_", ""): load(open(f"{path}{f}.pkl", "rb"))
    for f in file_names
}

In [692]:
dfs_backtest = {
    k: create_backtest_dataset(v)
    for k, v in results.items()
}

#### Cumulative profits

In [693]:
initial_invest = 100

In [694]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold_returns, 
    initial_invest, 
    layout)

fig.show()

#### Analysis per trading window for one setting

In [695]:
setting = "rrl_no_fees_3"

df = dfs_backtest[setting]
batch_names = np.unique(df["Batch"])

In [696]:
metric ="Cumulative returns"
fig_title = f"{metric} per trading window | {setting}"

ncols = 4
nrows = generate_nrows(len(batch_names), ncols)

titles = generate_subplot_titles(batch_names)

fig = init_subplots(nrows, ncols, titles)
fig = populate_subplots(fig, df, batch_names, titles, ncols, y_varname=metric, initial_invest=initial_invest)

layout = generate_layout(
    fig_height=1200, 
    fig_width=1400, 
    fig_title=fig_title)
    
fig.update_layout(layout)

In [697]:
df_barplot = generate_df_barplot(df)
title = f"% Cumulative returns at the end of each trading window | {setting}"

layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title=title,
    x_axis_args={"title": {"text": "Trading window"}}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "%"}}
)
fig = make_cumrets_barplot(df_barplot, layout)

fig.show()

#### Portfolio allocation

In [725]:
assets = results[setting]["assets"]
positions = results[setting]["positions"]

avg_positions_per_window = np.array([p.mean(axis=1) for p in positions]) 
avg_positions = avg_positions_per_window.mean(axis=0)


df_pie = pd.DataFrame({
    "Asset": assets, 
    "% Share": 100 * avg_positions
})

In [726]:
import plotly.express as px

palette = px.colors.sequential.Blues[-len(assets):]
palette.reverse()

fig = px.pie(
    df_pie, 
    values="% Share", 
    names="Asset", 
    hole=.4, 
    color_discrete_sequence=palette)

layout = generate_layout(
    fig_height=500, 
    fig_width=700, 
    fig_title="Average portfolio allocation over trading windows"
)
fig.update_layout(layout)

fig.show()